In [0]:
!apt-get install -y -qq software-properties-common module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131323 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&acces

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
root = '/content/drive/'

In [0]:
import os
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from numpy.random import permutation
from keras.optimizers import SGD
import pandas as pd
import datetime
import glob
import cv2
import math
import pickle
from collections import OrderedDict
from keras import backend as K

Using TensorFlow backend.


Data set constants:

In [0]:
activities = ['a' +  "{:02d}".format(x) for x in range(1, 17)]
print(activities)

['a01', 'a02', 'a03', 'a04', 'a05', 'a06', 'a07', 'a08', 'a09', 'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16']


Source: https://github.com/yakzan/Human-Action-Recognition-with-Keras

In [0]:
weights_file = 'vgg16_weights.h5'
top_model_weights_file = 'fc_model.h5'
whole_model_weights_path = root + 'whole_model.h5'

train_data_dir = root + 'msrda/msrda_rgb/train/'
test_data_dir = root + 'msrda/msrda_rgb/test/'
test_images_path = root + 'msrda/msrda_rgb/test/test/'

Features of the data set:

In [0]:
nb_test_samples = len(os.listdir(os.chdir(test_data_dir)))

nb_train_samples = 0
for act in activities:
    nb_train_samples += len(os.listdir(os.chdir(train_data_dir + act)))
    
print(nb_test_samples)
print(nb_train_samples)

601
1623


In [0]:
img_width, img_height = 640, 480
color_type_global = 3

In [0]:
# You can set a larger value here, according with the memory of your GPU.
batch_size = 16
nb_epoch = 100

Load the weights of the pre-trained VGG16:

In [0]:
from keras.applications.vgg16 import VGG16
from keras.layers import Input

model = VGG16(include_top=False, weights='imagenet')
image_input = Input(shape=(480, 640, 3), name='image_input')
output_vgg16_conv = model(image_input)

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 2s 0us/step


Add the fully-connected layers:

In [0]:
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(16, activation='softmax', name='predictions')(x)

Create own model:

In [0]:
from keras.models import Model

my_model = Model(input=image_input, output=x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("im..., outputs=Tensor("pr...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 480, 640, 3)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 153600)            0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              629149696 
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
predictions (Dense)          (None, 16)                65552     
Total params: 660,711,248
Trainable params: 660,711,248
Non-trainable params: 0
______________________________________________________________

In [0]:
# Set the first 15 layers to non-trainable (the original weights will not be updated).
for layer in my_model.layers[:15]:
    layer.trainable = False

Compile the model with a SGD/momentum optimizer:

In [0]:
my_model.compile(loss = "categorical_crossentropy",
              optimizer=optimizers.SGD(lr=1e-6, momentum=0.9),
              metrics=['mean_squared_logarithmic_error', 'accuracy'])

Data augmentation:

In [0]:
train_datagen = ImageDataGenerator(shear_range=0.3, zoom_range=0.3, rotation_range=0.3)
test_datagen = ImageDataGenerator()

Training:

In [0]:
train_generator = train_datagen.flow_from_directory(
        directory=train_data_dir,
        target_size=(img_height, img_width),
        batch_size=16,
        class_mode='categorical')

Found 1623 images belonging to 16 classes.


In [0]:
class_dictionary = train_generator.class_indices
sorted_class_dictionary = OrderedDict(sorted(class_dictionary.items()))
sorted_class_dictionary = sorted_class_dictionary.values()
print(sorted_class_dictionary)

odict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])


Fine-tuning the model:

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=3, mode='min', verbose=1)
checkpoint = ModelCheckpoint('model_best_weights.h5', monitor='loss', verbose=1, save_best_only=True, mode='min', period=1)
                             
my_model.fit_generator(
        train_generator,
        steps_per_epoch = int(nb_train_samples / batch_size),
        epochs = nb_epoch,
        validation_data = train_generator,
        validation_steps= int(nb_train_samples / batch_size),
        callbacks = [early_stop, checkpoint])

Epoch 1/100
101/101 [==============================] - 741s 7s/step - loss: 11.6961 - mean_squared_logarithmic_error: 0.0528 - acc: 0.0520 - val_loss: 11.6296 - val_mean_squared_logarithmic_error: 0.0523 - val_acc: 0.0510

Epoch 00001: loss improved from inf to 11.69112, saving model to model_best_weights.h5
Epoch 2/100
101/101 [==============================] - 848s 8s/step - loss: 11.6417 - mean_squared_logarithmic_error: 0.0527 - acc: 0.0526 - val_loss: 11.5348 - val_mean_squared_logarithmic_error: 0.0522 - val_acc: 0.0554

Epoch 00002: loss improved from 11.69112 to 11.64868, saving model to model_best_weights.h5
Epoch 3/100
101/101 [==============================] - 827s 8s/step - loss: 11.6415 - mean_squared_logarithmic_error: 0.0523 - acc: 0.0569 - val_loss: 11.6432 - val_mean_squared_logarithmic_error: 0.0519 - val_acc: 0.0641

Epoch 00003: loss improved from 11.64868 to 11.64477, saving model to model_best_weights.h5
Epoch 4/100
101/101 [==============================] - 818s 